* Take escape embeddings format target, and the sars-cov2-spike fasta sequence from Cao (downloaded seperately), and output csv with all sequence variants, and escaper score

* Data/labels from: https://github.com/swansonk14/escape_embeddings
* Fasta was downloaded fom: https://github.com/jbloomlab/SARS2_RBD_Ab_escape_maps/tree/main/data/2022_Cao_Omicron
    * `https://github.com/jbloomlab/SARS2_RBD_Ab_escape_maps/blob/main/data/2022_Cao_Omicron/spike.fasta`
    * * Note: They  did mutations on the DOMAIN
      *  Cao processing: https://github.com/jbloomlab/SARS2_RBD_Ab_escape_maps/blob/main/data/2022_Cao_Omicron/process_data.ipynb
      *  Are we sure this is the right sequence???
  *  `condition` = ANTIBODY_CONDITION

In [1]:
import pandas as pd
from Bio import SeqIO


In [2]:
# Function to replace character in a sequence ; assuming 0 vs 1 indexed also
# def replace_char(seq, site, mutation,offset_index=0):
#     return seq[:site] + mutation + seq[site + offset_index:]
def replace_char(seq, site, mutation,offset_index=1):
    return seq[:site- offset_index] + mutation + seq[site :]

In [3]:
inputSeqFile = open("Cao_Omicron_spike.fasta")
SeqDict = SeqIO.to_dict(SeqIO.parse(inputSeqFile, "fasta"))
print(SeqDict)
inputSeqFile.close()

a = SeqDict['QHD43416.1'].seq.strip()
sequence = "".join(a)
print(len(sequence))
print(sequence)

{'QHD43416.1': SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='QHD43416.1', name='QHD43416.1', description='QHD43416.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]', dbxrefs=[])}
1273
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSN

#### note: 1 vs 0 indexing? !
* Also: dropping condition greatly changes # results!! ;
* * Condition = antibody condition?

In [4]:
df = pd.read_csv("Escape Embeddings/data/data.csv").drop_duplicates()
print((df["mut_escape"]>0).value_counts())
df

mut_escape
False    962282
True      30658
Name: count, dtype: int64


,condition,site,wildtype,mutation,mut_escape
0,1-57,331,N,A,0.0
1,1-57,331,N,C,0.0
2,1-57,331,N,D,0.0
3,1-57,331,N,E,0.0
4,1-57,331,N,F,0.0
...,...,...,...,...,...
992935,WIBP-2B11,531,T,S,0.0
992936,WIBP-2B11,531,T,T,0.0
992937,WIBP-2B11,531,T,V,0.0
992938,WIBP-2B11,531,T,W,0.0


In [5]:
df.condition.value_counts()

condition
1-57         4020
COV2-2130    4020
COV2-2268    4020
COV2-2308    4020
COV2-2353    4020
             ... 
BD55-3433    4020
BD55-3440    4020
BD55-3451    4020
BD55-3457    4020
WIBP-2B11    4020
Name: count, Length: 247, dtype: int64

In [6]:
df.drop(columns=["condition"]).drop_duplicates().shape[0]

34678

In [7]:
df.drop(columns=["condition"]).drop_duplicates()

,site,wildtype,mutation,mut_escape
0,331,N,A,0.000000
1,331,N,C,0.000000
2,331,N,D,0.000000
3,331,N,E,0.000000
4,331,N,F,0.000000
...,...,...,...,...
992092,489,Y,R,0.001041
992093,489,Y,S,0.185581
992094,489,Y,T,0.088192
992095,489,Y,V,0.014363


In [8]:
df.drop(columns=["condition"]).drop_duplicates(subset=["site","mutation"])

,site,wildtype,mutation,mut_escape
0,331,N,A,0.0
1,331,N,C,0.0
2,331,N,D,0.0
3,331,N,E,0.0
4,331,N,F,0.0
...,...,...,...,...
4015,531,T,S,0.0
4016,531,T,T,0.0
4017,531,T,V,0.0
4018,531,T,W,0.0


In [9]:
df.site.agg(["min","max","count"])

min         331
max         531
count    992940
Name: site, dtype: int64

In [10]:
df.loc[df["mut_escape"]==0]

,condition,site,wildtype,mutation,mut_escape
0,1-57,331,N,A,0.0
1,1-57,331,N,C,0.0
2,1-57,331,N,D,0.0
3,1-57,331,N,E,0.0
4,1-57,331,N,F,0.0
...,...,...,...,...,...
992935,WIBP-2B11,531,T,S,0.0
992936,WIBP-2B11,531,T,T,0.0
992937,WIBP-2B11,531,T,V,0.0
992938,WIBP-2B11,531,T,W,0.0


In [11]:
sequence[329:333]

'PNIT'

In [12]:
df.drop_duplicates("site")

,condition,site,wildtype,mutation,mut_escape
0,1-57,331,N,A,0.0
20,1-57,332,I,A,0.0
40,1-57,333,T,A,0.0
60,1-57,334,N,A,0.0
80,1-57,335,L,A,0.0
...,...,...,...,...,...
3920,1-57,527,P,A,0.0
3940,1-57,528,K,A,0.0
3960,1-57,529,K,A,0.0
3980,1-57,530,S,A,0.0


### make subset that ignores antibody: max antibody score/condition per site
* Could do max... o min ,avg... 

In [13]:
df = df.drop(columns="condition").drop_duplicates()
print(df.shape[0])
print(df.nunique())
print("Aggregatign and ignore antibody/condition:")
df = df.groupby(['site', 'wildtype', 'mutation']).max().reset_index()
print(df.shape[0])
print(df.nunique())
df["mut_escape_class"] = (df["mut_escape"]>0).astype(int)
print(">0:",df["mut_escape_class"].agg(["mean","sum"]).round(2))

df

34678
site            201
wildtype         19
mutation         20
mut_escape    30606
dtype: int64
Aggregatign and ignore antibody/condition:
4020
site           201
wildtype        19
mutation        20
mut_escape    2119
dtype: int64
>0: mean       0.53
sum     2123.00
Name: mut_escape_class, dtype: float64


,site,wildtype,mutation,mut_escape,mut_escape_class
0,331,N,A,0.009428,1
1,331,N,C,0.000000,0
2,331,N,D,0.026596,1
3,331,N,E,0.112258,1
4,331,N,F,0.012256,1
...,...,...,...,...,...
4015,531,T,S,0.010178,1
4016,531,T,T,0.000000,0
4017,531,T,V,0.009521,1
4018,531,T,W,0.008336,1


In [14]:
df["seq"] = sequence
print(df["seq"].head(2).str[329:333])
print(df.head(3).apply(lambda row: replace_char(row['seq'], row['site'], row['mutation']), axis=1).str[329:333]) ## check that N was changed

df['seq'] = df.apply(lambda row: replace_char(row['seq'], row['site'], row['mutation']), axis=1)

df

0    PNIT
1    PNIT
Name: seq, dtype: object
0    PAIT
1    PCIT
2    PDIT
dtype: object


,site,wildtype,mutation,mut_escape,mut_escape_class,seq
0,331,N,A,0.009428,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,331,N,C,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,331,N,D,0.026596,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,331,N,E,0.112258,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,331,N,F,0.012256,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...,...,...,...
4015,531,T,S,0.010178,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4016,531,T,T,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4017,531,T,V,0.009521,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4018,531,T,W,0.008336,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [ ]:
df.to_csv("cao_escaper_targets_max.csv",index=F)